In [1]:
import pandas as pd
from utils.esutils import query_enterprises_scroll
import utils.threads as threads
import re
import utils.processes as processes

In [2]:
enterprises = query_enterprises_scroll(province="江苏省")

已加载 100000 条记录...
共加载 100000 条记录


In [5]:
df = pd.DataFrame(enterprises)
df.to_csv('江苏省企业.csv', index=False)

In [6]:
df = pd.read_csv('江苏省企业.csv')

In [16]:
def deal(row):
        address = row["registered_address"]
        province = row["province"]
        city = row["city"]
        district = row["district"]

        row["extra_address"] = None
        row["matched_province"] = None
        row["matched_city"] = None
        row["matched_district"] = None

        if pd.isna(address) or address == "-":
            return row

        extra_address = address
        matched_province = None
        matched_city = None
        matched_district = None
        matched_town_or_street = None

        if not pd.isna(province) and province != "-" and province in extra_address:
            matched_province = province
            extra_address = extra_address.replace(province, "", 1)

        if not pd.isna(city) and city != "-" and city in extra_address:
            matched_city = city
            extra_address = extra_address.replace(city, "", 1)

        if not pd.isna(district) and district != "-" and district in extra_address:
            matched_district = district
            extra_address = extra_address.replace(district, "", 1)
            
        # 提取开发区
        town_or_street_pattern = r"[^\s]{2,10}(?:镇|街道)"
        match = re.search(town_or_street_pattern, extra_address)
        if match:
            matched_town_or_street = match.group(0)
            extra_address = extra_address.replace(matched_town_or_street, "", 1)
            
        # 提取镇/街道（以“镇”或“街道”结尾的词）
        town_or_street_pattern = r"[^\s]{2,10}(?:镇|街道)"
        match = re.search(town_or_street_pattern, extra_address)
        if match:
            matched_town_or_street = match.group(0)
            extra_address = extra_address.replace(matched_town_or_street, "", 1)

        extra_address = extra_address.strip()
        if not extra_address:
            extra_address = None

        row["extra_address"] = extra_address
        row["matched_province"] = matched_province
        row["matched_city"] = matched_city
        row["matched_district"] = matched_district
        row["matched_town"] = matched_town_or_street

        return row

In [17]:
ok_df = processes.process_dataframe_multiprocess(df, deal, num_processes=4, chunk_size=100000)

处理进度 2025-04-15 20:56:58:0


In [22]:
def csv_to_bio(df, output_file):
    selected_columns = ["province", "city", "district", "dev", "matched_town", "extra_address"]
    
    with open(output_file, "w", encoding="utf-8") as f:
        for index, row in df.iterrows():
            # 处理每个字段
            for col in selected_columns:
                if pd.notna(row[col]) and (col != "matched_town" or row[col]):  # 跳过空的 matched_town
                    chars = list(row[col])  # 将字符串拆分为字符列表
                    if col == "province":
                        tag_prefix = "PROV"
                    elif col == "city":
                        tag_prefix = "CITY"
                    elif col == "district":
                        tag_prefix = "DISTRICT"
                    elif col == "dev":
                        tag_prefix = "DEV"
                    elif col == "matched_town":
                        tag_prefix = "TOWN"
                    elif col == "extra_address":
                        tag_prefix = "EXTRA"
                    
                    # 分配BIO标签
                    for i, char in enumerate(chars):
                        if i == 0:
                            f.write(f"{char} B-{tag_prefix}\n")
                        elif i == len(chars) - 1:
                            f.write(f"{char} E-{tag_prefix}\n")
                        else:
                            f.write(f"{char} I-{tag_prefix}\n")
            # 每条地址后加空行
            if index < len(df) - 1:
                f.write("\n")

# 生成train.txt
csv_to_bio(ok_df.head(100000), "test.txt")

In [21]:
ok_df.head(100000).to_csv('test.csv', index=False)